# Video Player

## How to use Video Player
- Video

```
player = VideoPlayer(source_path, verbose)
player.play(start, step, last)
```

- WebCam

```
player = VideoPlayer(0)
player.play()
```

In [1]:
import cv2
import keras
from keras.applications.imagenet_utils import preprocess_input
from keras.backend.tensorflow_backend import set_session
from keras.models import Model
from keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np
from imageio import imread
import tensorflow as tf

from ssd import SSD300
from ssd_utils import BBoxUtility

Using TensorFlow backend.


In [2]:
class VideoPlayer:
    def __init__(self, source_path=None, verbose=1):
        """
        source_path: file path
          if nothing specified 0 is set and WebCam is selected if available.
        verbose: to be used to karas prediction (0,1)
        ex)
        video = VideoPlayer(source_pathpath) # for video
        video.play(10,10,1000) # view from 10 frame to 1000 frame step 10
        video = VideoPlayer() # for webcam
        video.play()
        To interrupt video, push escape key.
        """
        if source_path == None:
            self.source_path = 0
        else:
            self.source_path = source_path
        self.cap = cv2.VideoCapture(self.source_path)
        self.verbose = verbose
        self.frame_count = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))
        self.fps = int(self.cap.get(cv2.CAP_PROP_FPS))
        self.width = int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        self.height = int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        #self.voc_classes = ['worker', 'others']
        self.voc_classes = ['Aeroplane', 'Bicycle', 'Bird', 'Boat', 'Bottle',
               'Bus', 'Car', 'Cat', 'Chair', 'Cow', 'Diningtable',
               'Dog', 'Horse','Motorbike', 'Person', 'Pottedplant',
               'Sheep', 'Sofa', 'Train', 'Tvmonitor']
        self.num_classes = len(self.voc_classes) + 1
        self.input_shape = (300, 300, 3)
        self.model = SSD300(self.input_shape, num_classes=self.num_classes)
        #self.model.load_weights('custome_weights.23-1.22.hdf5', by_name=True)
        self.model.load_weights('weights_SSD300.hdf5', by_name=True)
        self.bbox_util = BBoxUtility(self.num_classes)
        self.frame = None
        self.idx = None
        
        print('width: {}, height: {}'.format(self.width, self.height))
        print('Total frames: {}'.format(self.frame_count))
        print('fps: {}'.format(self.fps))

    def _play(self):
        
        img = cv2.resize(self.frame, (300, 300))
        pred = self.model.predict(np.array([img]), batch_size=1, verbose=self.verbose)
        results = self.bbox_util.detection_out(pred)
        
        det_label = results[0][:, 0]
        det_conf = results[0][:, 1]
        det_xmin = results[0][:, 2]
        det_ymin = results[0][:, 3]
        det_xmax = results[0][:, 4]
        det_ymax = results[0][:, 5]
        
        # Get detections with confidence higher than 0.6.
        top_indices = [i for i, conf in enumerate(det_conf) if conf >= 0.6]
        
        top_conf = det_conf[top_indices]
        top_label_indices = det_label[top_indices].tolist()
        top_xmin = det_xmin[top_indices]
        top_ymin = det_ymin[top_indices]
        top_xmax = det_xmax[top_indices]
        top_ymax = det_ymax[top_indices]
        
        #colors = plt.cm.hsv(np.linspace(0, 1, 21)).tolist()
        font = cv2.FONT_HERSHEY_PLAIN
        
        if self.frame_count != -1: 
            display_frm = 'frame: {}'.format(self.idx)
            cv2.putText(self.frame, display_frm, (0, 25), font, 1, (255,255,255), 1)
        
        for i in range(top_conf.shape[0]):
            xmin = int(round(top_xmin[i] * self.frame.shape[1]))
            ymin = int(round(top_ymin[i] * self.frame.shape[0]))
            xmax = int(round(top_xmax[i] * self.frame.shape[1]))
            ymax = int(round(top_ymax[i] * self.frame.shape[0]))
            score = top_conf[i]
            label = int(top_label_indices[i])
            label_name = self.voc_classes[label - 1]
            display_txt = '{:0.2f}, {}'.format(score, label_name)
            
            cv2.rectangle(self.frame, (xmin, ymin), (xmax, ymax), (0,255,0), 2)
            cv2.putText(self.frame, display_txt, (xmin, ymin), font, 1, (0,0,255), 1)
        
        #cv2.imshow("img", self.frame)


    def play(self, start=0, step=100, last=None):
        """
        these parameters is available only for video
        
        start: the frame to start you want
        step: interval of time between showing
        last: the frame to last you want
        """
        
        if self.frame_count == -1:
            while(1):
                ret, self.frame = self.cap.read()
                if not ret:
                    print("Video Reader does not work well.")
                    break
                self._play()
                cv2.imshow('img', self.frame)
                if cv2.waitKey(1) == 27: # escape
                    print("Breaked.")
                    break
        else:
            if last == None:
                last = self.frame_count
            
            for self.idx in range(start, last, step):
                self.cap.set(cv2.CAP_PROP_POS_FRAMES, self.idx)
                ret, self.frame = self.cap.read()
                if not ret:
                    print("Video Reader does not work well.")
                    break
                self._play()
                cv2.imshow('img', self.frame)
                if cv2.waitKey(1) == 27: # escape
                    print("Breaked.")
                    break
        self.cap.release()
        cv2.destroyAllWindows()
        print("Finished.")

In [3]:
# webcam mode
player = VideoPlayer(verbose=0)

width: 640, height: 480
Total frames: -1
fps: 0


In [4]:
player.play()

Breaked.
Finished.


# Video Recorder

## How to use Video Recorder
- Video

```
recorder = VideoRecorder(source_path, verbose)
recorder.record(target_path, start, step, last)
```

- WebCam

```
recorder = VideoRecorder(0, verbose)
recorder.record(target_path)
```

In [5]:
class VideoRecorder(VideoPlayer):
    
    def record(self, target_path=None, start=0, step=100, last=None):
        """
        To interrupt video, push escape key.
        these parameters is available only for video

        start: the frame to start you want
        step: interval of time between showing
        last: the frame to last you want
        """
        
        if target_path == None:
            target_path = 'out.mp4'
        
        if self.fps == 0:
            self.fps = 10
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        out = cv2.VideoWriter(target_path, fourcc, self.fps, (self.width, self.height))
        
        if self.frame_count == -1:
            while(1):
                ret, self.frame = self.cap.read()
                if not ret:
                    print("Video Reader does not work well.")
                    break
                self._play()
                cv2.imshow('img', self.frame)
                out.write(self.frame)
                if cv2.waitKey(1) == 27: # escape
                    print("Breaked.")
                    break
        else:
            if last == None:
                last = self.frame_count
            
            for self.idx in range(start, last, step):
                self.cap.set(cv2.CAP_PROP_POS_FRAMES, self.idx)
                ret, self.frame = self.cap.read()
                if not ret:
                    print("Video Reader does not work well.")
                    break
                self._play()
                out.write(self.frame)
                if cv2.waitKey(1) == 27: # escape
                    print("Breaked.")
                    break
        self.cap.release()
        cv2.destroyAllWindows()
        print("Finished.")

In [6]:
# webcam mode
recorder = VideoRecorder(0, verbose=0)

width: 640, height: 480
Total frames: -1
fps: 0


In [7]:
recorder.record('out.avi')

Breaked.
Finished.
